In [1]:
# Experiment 2 Script for COVID-19 Forecasting
# Change path to the root directory of the project
import os
os.chdir("../../")

# Description: This script contains the code for the second experiment in the project, 
# forecasting COVID-19 MVBeds using various RNN models and hyperparameter tuning with Simulated Annealing.

# Imports for handling data
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import cycle

# Imports for machine learning
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# Imports for visualization
import plotly.express as px
import plotly.graph_objects as go

# Progress bar
from tqdm.autonotebook import tqdm
tqdm.pandas()

# Local imports for data loaders and models
from src.utils import plotting_utils
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.multivariate_models import SingleStepRNNConfig, SingleStepRNNModel, Seq2SeqConfig, Seq2SeqModel, RNNConfig
from src.transforms.target_transformations import AutoStationaryTransformer

# Set seeds for reproducibility
pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

torch.set_float32_matmul_precision('high')

# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import logging

# Set logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


Global seed set to 42


In [2]:
# Utility Functions

def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig

def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample

def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)

def plot_forecast(pred_df, forecast_columns, selected_area, forecast_display_names=None, save_path=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.D3  # Use a colorblind-friendly palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines+markers",
            marker=dict(size=8, opacity=0.7, symbol='circle'),
            line=dict(color=act_color, width=3),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    line_styles = ["solid", "dash", "dot", "dashdot"]
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']
    for col, display_col, line_style, marker in zip(forecast_columns, forecast_display_names, cycle(line_styles), cycle(markers)):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=6, symbol=marker),
                line=dict(color=next(colors), width=2, dash=line_style),
                name=display_col,
            )
        )

    fig.update_layout(
        title=f"COVID-19 MVBeds Forecast Comparison for {selected_area}",
        title_font=dict(size=20),
        xaxis_title="Date",
        yaxis_title="COVID-19 MVBeds",
        xaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        yaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        legend=dict(
            font=dict(size=10),
            orientation="h",
            yanchor="top",
            y=1.02,  # Move the legend down to separate it from the title
            xanchor="center",
            x=0.5
        ),
        template="plotly_white",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=10, r=40, t=80, b=40),  # Adjust left margin for better alignment
        width=1200,
        height=600
    )

    if save_path:
        pio.write_image(fig, save_path)
    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

# Function to convert data to float32
def to_float32(df):
    for col in df.columns:
        df[col] = df[col].astype("float32")
    return df

In [3]:
# Load and Prepare Data
data_path = Path("data/hos_data/england_data.csv")
# data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data = pd.read_csv(data_path)
data["date"] = pd.to_datetime(data["date"])

# Select and Process Data
selected_area = "England"
# data_filtered = data[data["areaName"] == selected_area]
data_filtered = data.copy()
# Data Processing
data_filtered["date"] = pd.to_datetime(data_filtered["date"])
# data_filtered.sort_values(by=["date", "areaName"], inplace=True)
data_filtered.drop(
    [
        # "areaName",
        # "areaCode",
        "cumAdmissions",
        "cumulative_confirmed",
        "cumulative_deceased",
        "population",
        "latitude",
        "longitude",
        "epi_week",
        "region",
    ],
    axis=1,
    inplace=True,
)
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 888 non-null    datetime64[ns]
 1   covidOccupiedMVBeds  888 non-null    float64       
 2   hospitalCases        888 non-null    float64       
 3   newAdmissions        888 non-null    int64         
 4   new_confirmed        888 non-null    float64       
 5   new_deceased         888 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 41.8 KB


In [4]:
# Add rolling features
def add_rolling_features(df, window_size, columns, agg_funcs=None):
    if agg_funcs is None:
        agg_funcs = ["mean"]
    added_features = {}
    for column in columns:
        for func in agg_funcs:
            roll_col_name = f"{column}_rolling_{window_size}_{func}"
            df[roll_col_name] = df[column].rolling(window_size).agg(func)
            if column not in added_features:
                added_features[column] = []
            added_features[column].append(roll_col_name)
    df.dropna(inplace=True)
    return df, added_features


window_size = 7
columns_to_roll = ["hospitalCases", "newAdmissions", "new_confirmed", "new_deceased"]
agg_funcs = ["mean", "std"]

data_filtered, added_features = add_rolling_features(
    data_filtered, window_size, columns_to_roll, agg_funcs
)

for column, features in added_features.items():
    logging.info(f"{column}: {', '.join(features)}")

2024-05-22 22:06:44,680 - INFO - hospitalCases: hospitalCases_rolling_7_mean, hospitalCases_rolling_7_std
2024-05-22 22:06:44,681 - INFO - newAdmissions: newAdmissions_rolling_7_mean, newAdmissions_rolling_7_std
2024-05-22 22:06:44,682 - INFO - new_confirmed: new_confirmed_rolling_7_mean, new_confirmed_rolling_7_std
2024-05-22 22:06:44,682 - INFO - new_deceased: new_deceased_rolling_7_mean, new_deceased_rolling_7_std


In [5]:
# Add time-lagged features
def add_lags(data, lags, features):
    added_features = []
    for feature in features:
        for lag in lags:
            new_feature = feature + f"_lag_{lag}"
            data[new_feature] = data[feature].shift(lag)
            added_features.append(new_feature)
    return data, added_features


lags = [1, 2, 3, 5, 7, 14, 21]
data_filtered, added_features = add_lags(data_filtered, lags, ["covidOccupiedMVBeds"])
data_filtered.dropna(inplace=True)


# Create temporal features
def create_temporal_features(df, date_column):
    df["month"] = df[date_column].dt.month
    df["day"] = df[date_column].dt.day
    df["day_of_week"] = df[date_column].dt.dayofweek
    return df


data_filtered = create_temporal_features(data_filtered, "date")
data_filtered = data_filtered.set_index("date")

In [6]:
# Load and process the SEIRD data
seird_data = pd.read_csv(f"reports/output/pinn_{selected_area}_output.csv")
seird_data["date"] = pd.to_datetime(seird_data["date"])
seird_data.set_index("date", inplace=True)

# Merge the two dataframes on the date index
merged_data = pd.merge(
    data_filtered, seird_data, left_index=True, right_index=True, how="inner"
)

# Drop rows with any missing values
merged_data.dropna(inplace=True)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 392 entries, 2020-05-05 to 2021-05-31
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   covidOccupiedMVBeds           392 non-null    float64
 1   hospitalCases                 392 non-null    float64
 2   newAdmissions                 392 non-null    int64  
 3   new_confirmed                 392 non-null    float64
 4   new_deceased                  392 non-null    float64
 5   hospitalCases_rolling_7_mean  392 non-null    float64
 6   hospitalCases_rolling_7_std   392 non-null    float64
 7   newAdmissions_rolling_7_mean  392 non-null    float64
 8   newAdmissions_rolling_7_std   392 non-null    float64
 9   new_confirmed_rolling_7_mean  392 non-null    float64
 10  new_confirmed_rolling_7_std   392 non-null    float64
 11  new_deceased_rolling_7_mean   392 non-null    float64
 12  new_deceased_rolling_7_std    392 non-null   

In [7]:
merged_data.head()

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,hospitalCases_rolling_7_mean,hospitalCases_rolling_7_std,newAdmissions_rolling_7_mean,newAdmissions_rolling_7_std,new_confirmed_rolling_7_mean,...,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week,susceptible,active_cases,recovered,cumulative_deceased
date,,,,,,,,,,,,,,,,,,,,,
2020-05-05,1662.0,11754.0,959,3060.0,390.0,12425.142857,523.583304,1092.714286,124.451750,3416.285714,...,2093.0,2701.0,2849.0,5,5,1,56021030.0,124772.85,-7095.0913,26125.190
2020-05-06,1610.0,11227.0,1040,3191.0,390.0,12139.857143,558.643425,1056.285714,87.081790,3213.571429,...,2045.0,2646.0,2847.0,5,6,2,56018188.0,127764.14,-7550.1600,26483.633
2020-05-07,1493.0,10726.0,998,3112.0,394.0,11826.142857,654.576563,1023.571429,45.206405,3053.714286,...,1991.0,2563.0,2838.0,5,7,3,56015530.0,129039.28,-6299.3315,26851.654
2020-05-08,1446.0,10249.0,743,2571.0,330.0,11487.000000,776.541048,975.285714,109.059180,2846.142857,...,1954.0,2414.0,2780.0,5,8,4,56013010.0,128682.49,-3349.1306,27229.890
2020-05-09,1398.0,10176.0,798,1767.0,328.0,11181.571429,816.484508,949.285714,127.827637,2716.142857,...,1844.0,2405.0,2868.0,5,9,5,56010556.0,126888.50,1115.4313,27615.020


In [8]:
# plot cumulative_deceased 
# fig = px.line(merged_data, x=merged_data.index, y="susceptible", title="Cumulative Deceased Cases")
# fig.show()

In [9]:
# Set the target variable and make it stationary
target = "covidOccupiedMVBeds"
seasonal_period = 7
auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period)
data_stat = auto_stationary.fit_transform(merged_data[[target]], freq="D")
merged_data[target] = data_stat.values

# Ensure the index is a DateTimeIndex
merged_data.index = pd.to_datetime(merged_data.index)

In [10]:
# Filter data between the specified dates
start_date = "2020-04-27"
end_date = "2021-05-31"
merged_data = merged_data[start_date:end_date]

min_date = merged_data.index.min()
max_date = merged_data.index.max()

# Calculate the range of dates
date_range = max_date - min_date
logging.info(f"Data ranges from {min_date} to {max_date} ({date_range.days} days)")

2024-05-22 22:06:45,565 - INFO - Data ranges from 2020-05-05 00:00:00 to 2021-05-31 00:00:00 (391 days)


In [12]:
# # Calculate split points
# train_end = min_date + pd.DateOffset(months=9)
# val_end = train_end + pd.DateOffset(months=2)
# test_end = val_end + pd.DateOffset(months=1)

# train = merged_data[merged_data.index <= train_end]
# val = merged_data[(merged_data.index > train_end) & (merged_data.index <= val_end)]
# test = merged_data[merged_data.index > val_end]

# Calculate split points
total_days = date_range.days
train_end = min_date + pd.Timedelta(days=int(total_days * 0.70))
val_end = train_end + pd.Timedelta(days=int(total_days * 0.20))

# Split the data into training, validation, and testing sets
train = merged_data[merged_data.index <= train_end]
val = merged_data[(merged_data.index > train_end) & (merged_data.index <= val_end)]
test = merged_data[merged_data.index > val_end]

# Calculate the percentage of dates in each dataset
total_sample = len(merged_data)
train_sample = len(train) / total_sample * 100
val_sample = len(val) / total_sample * 100
test_sample = len(test) / total_sample * 100

print(f"Train: {train_sample:.2f}%, Validation: {val_sample:.2f}%, Test: {test_sample:.2f}%")
print(f"Train: {len(train)} samples, Validation: {len(val)} samples, Test: {len(test)} samples")
print(f"Max date in train: {train.index.max()}, Min date in validation: {val.index.min()}, Max date in test: {test.index.max()}")

Train: 69.90%, Validation: 19.90%, Test: 10.20%
Train: 274 samples, Validation: 78 samples, Test: 40 samples
Max date in train: 2021-02-02 00:00:00, Min date in validation: 2021-02-03 00:00:00, Max date in test: 2021-05-31 00:00:00


In [19]:
# Concatenate the DataFrames
sample_df = pd.concat([train, val, test])

# Convert all the feature columns to float32
sample_df = to_float32(sample_df)

columns_to_select = [
    "covidOccupiedMVBeds",
    # "hospitalCases_rolling_7_mean",
    # "hospitalCases_rolling_7_std",
    # "newAdmissions_rolling_7_mean",
    # "newAdmissions_rolling_7_std",
    # "new_confirmed_rolling_7_mean",
    # "new_confirmed_rolling_7_std",
    # "new_deceased_rolling_7_mean",
    # "new_deceased_rolling_7_std",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
    # "susceptible",
    # "active_cases",
    # "recovered",
    # "cumulative_deceased",
]

pinn_features = [
    "covidOccupiedMVBeds",
    "hospitalCases_rolling_7_mean",
    "hospitalCases_rolling_7_std",
    "newAdmissions_rolling_7_mean",
    "newAdmissions_rolling_7_std",
    "new_confirmed_rolling_7_mean",
    "new_confirmed_rolling_7_std",
    "new_deceased_rolling_7_mean",
    "new_deceased_rolling_7_std",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
    "susceptible",
    "active_cases",
    "recovered",
    "cumulative_deceased",
]

sample_df = sample_df[columns_to_select]
cols = list(sample_df.columns)
cols.remove("covidOccupiedMVBeds")
sample_df = sample_df[cols + ["covidOccupiedMVBeds"]]
sample_df.head()

,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week,covidOccupiedMVBeds
date,,,,,,,,,,,
2020-05-05,1724.0,1795.0,1844.0,1991.0,2093.0,2701.0,2849.0,5.0,5.0,1.0,44.980167
2020-05-06,1662.0,1724.0,1795.0,1954.0,2045.0,2646.0,2847.0,5.0,6.0,2.0,44.580746
2020-05-07,1610.0,1662.0,1724.0,1844.0,1991.0,2563.0,2838.0,5.0,7.0,3.0,43.680965
2020-05-08,1493.0,1610.0,1662.0,1795.0,1954.0,2414.0,2780.0,5.0,8.0,4.0,43.300884
2020-05-09,1446.0,1493.0,1610.0,1724.0,1844.0,2405.0,2868.0,5.0,9.0,5.0,42.907440


In [20]:
# Prepare DataModule for PyTorch Lightning
datamodule = TimeSeriesDataModule(
    data=sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7,  # 7 days window
    horizon=1,  # single step
    normalize="global",  # normalizing the data
    batch_size=32,
    num_workers=0,
)
datamodule.setup()

# Initialize pred_df
pred_df = test.copy()

In [21]:
# Function to train and evaluate models
def train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df):
    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )
    trainer.fit(model, datamodule)

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean
    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    metrics = {
        "Algorithm": algorithm_name,
        "MAE": mae(actuals, predictions),
        "MSE": mse(actuals, predictions),
        "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
        "Forecast Bias": forecast_bias(actuals, predictions),
    }

    value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
    metrics = {
        key: format_.format(value)
        for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
    }

    pred_df_ = pd.DataFrame({algorithm_name: predictions}, index=test.index)
    pred_df = pred_df.join(pred_df_)
    metric_record.append(metrics)
    logging.info(metrics)
    return metric_record, pred_df

In [22]:
# Train Vanilla RNN
rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla RNN"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record=[], pred_df=pred_df)

# Train Vanilla LSTM
rnn_config = SingleStepRNNConfig(
    rnn_type="LSTM",
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla LSTM"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

# Train Vanilla GRU
rnn_config = SingleStepRNNConfig(
    rnn_type="GRU",
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla GRU"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 9.9 K 
1 | fc   | Linear  | 33    
2 | loss | MSELoss | 0     
---------------------------------
9.9 K     Trainable params
0         Non-trainable params
9.9 K     Total params
0.040     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-22 22:07:40,137 - INFO - {'Algorithm': 'Vanilla GRU', 'MAE': '17.0619', 'MSE': '306.9585', 'MASE': '67.7694', 'Forecast Bias': '17.06'}


In [23]:
metric_record

[{'Algorithm': 'Vanilla RNN',
  'MAE': '18.6227',
  'MSE': '360.5305',
  'MASE': '73.9691',
  'Forecast Bias': '18.62'},
 {'Algorithm': 'Vanilla LSTM',
  'MAE': '18.1847',
  'MSE': '341.1448',
  'MASE': '72.2292',
  'Forecast Bias': '18.18'},
 {'Algorithm': 'Vanilla GRU',
  'MAE': '17.0619',
  'MSE': '306.9585',
  'MASE': '67.7694',
  'Forecast Bias': '17.06'}]

In [24]:
# plot the forecast
forecast_columns = ["Vanilla RNN", "Vanilla LSTM", "Vanilla GRU"]
forecast_display_names = ["Vanilla RNN", "Vanilla LSTM", "Vanilla GRU"]
fig = plot_forecast(
    pred_df, forecast_columns, selected_area, forecast_display_names=forecast_display_names
)
fig.show()


In [18]:
param_bounds = {
    "rnn_type": ["RNN", "GRU", "LSTM"],
    "hidden_size": (32, 128),
    "num_layers": (5, 30),
    "bidirectional": [True, False]
}

initial_params = ["RNN", 32, 5, True]
initial_temp = 10

def objective(params):
    rnn_type, hidden_size, num_layers, bidirectional = params
    rnn_config = SingleStepRNNConfig(
        rnn_type=rnn_type,
        input_size=len(columns_to_select),
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional,
        learning_rate=1e-3
    )
    model = SingleStepRNNModel(rnn_config)
    model.float()

    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )
    trainer.fit(model, datamodule)
    
    shutil.rmtree("lightning_logs")

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean

    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    return np.mean(np.abs(actuals - predictions))

def neighbor(params):
    if len(params) == 4:
        rnn_type, hidden_size, num_layers, bidirectional = params
        hidden_size = np.random.randint(*param_bounds["hidden_size"])
        num_layers = np.random.randint(*param_bounds["num_layers"])
        rnn_type = np.random.choice(param_bounds["rnn_type"])
        bidirectional = bool(np.random.choice(param_bounds["bidirectional"]))
        return [rnn_type, hidden_size, num_layers, bidirectional]
    elif len(params) == 6:
        encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden = params
        hidden_size = np.random.randint(*param_bounds["hidden_size"])
        num_layers = np.random.randint(*param_bounds["num_layers"])
        encoder_type = np.random.choice(param_bounds["encoder_type"])
        decoder_type = np.random.choice(param_bounds["decoder_type"])
        bidirectional = bool(np.random.choice(param_bounds["bidirectional"]))
        decoder_use_all_hidden = bool(np.random.choice(param_bounds["decoder_use_all_hidden"]))
        return [encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden]

def simulated_annealing(objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate=0.20, verbose=True):
    current_params = initial_params
    current_cost = objective(current_params)
    best_params = current_params
    best_cost = current_cost
    temp = initial_temp
    cost_history = []

    for i in range(n_iter):
        candidate_params = neighbor(current_params)
        candidate_cost = objective(candidate_params)

        acceptance_probability = np.exp(-abs(candidate_cost - current_cost) / temp)

        if candidate_cost < current_cost or np.random.uniform() < acceptance_probability:
            current_params = candidate_params
            current_cost = candidate_cost

            if current_cost < best_cost:
                best_params = current_params
                best_cost = current_cost

        temp *= cooling_rate
        cost_history.append(best_cost)

        logging.info(f"Iteration: {i+1}, Best Cost: {best_cost:.4f}, Current Cost: {current_cost:.4f}, Temperature: {temp:.4f}")
        
        # Break early if the minimum cost has been constant for over 10 iterations
        if i > 20 and np.all(np.isclose(cost_history[-20:], cost_history[-1])):
            break

    return best_cost, best_params, cost_history

n_iter = 100
cooling_rate = 0.95

best_cost, best_params, cost_history = simulated_annealing(
    objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate
)

logging.info(f"Best Parameters: {best_params}, Best Cost: {best_cost}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(cost_history) + 1), y=cost_history, mode="lines"))
fig.update_layout(
    title="Simulated Annealing Optimization for Hyperparameter Tuning",
    xaxis_title="Iteration",
    yaxis_title="Best Cost",
    template="plotly_white",
)

fig.show()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 28.5 K
1 | fc   | Linear  | 65    
2 | loss | MSELoss | 0     
---------------------------------
28.5 K    Trainable params
0         Non-trainable params
28.5 K    Total params
0.114     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-22 21:51:57,373 - INFO - Iteration: 28, Best Cost: 5.8380, Current Cost: 13.5787, Temperature: 2.3783
2024-05-22 21:51:57,374 - INFO - Best Parameters: ['RNN', 86, 24, True], Best Cost: 5.838038189084196


In [19]:
nn_config = SingleStepRNNConfig(
    rnn_type=best_params[0],
    input_size=len(columns_to_select),
    hidden_size=best_params[1],
    num_layers=best_params[2],
    bidirectional=best_params[3],
    learning_rate=1e-3,
)

model = SingleStepRNNModel(rnn_config)
algorithm_name = f"Optimized {rnn_config.rnn_type} (SA)"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name | Type    | Params
---------------------------------
0 | rnn  | GRU     | 30.4 K
1 | fc   | Linear  | 33    
2 | loss | MSELoss | 0     
---------------------------------
30.5 K    Trainable params
0         Non-trainable params
30.5 K    Total params
0.122     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-22 21:51:59,399 - INFO - {'Algorithm': 'Optimized GRU (SA)', 'MAE': '16.5043', 'MSE': '298.7898', 'MASE': '65.6200', 'Forecast Bias': '16.50'}


In [20]:
HORIZON = 1
WINDOW = 7

encoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=True
)

decoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=True
)

rnn2fc_config = Seq2SeqConfig(
    encoder_type="LSTM",
    decoder_type="FC",
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=True,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)
algorithm_name = "Seq2Seq LSTM"
metric_record, pred_df = train_and_evaluate(model, rnn2fc_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type    | Params
------------------------------------
0 | encoder | LSTM    | 113 K 
1 | decoder | Linear  | 449   
2 | loss    | MSELoss | 0     
------------------------------------
114 K     Trainable params
0         Non-trainable params
114 K     Total params
0.457     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-22 21:52:05,093 - INFO - {'Algorithm': 'Seq2Seq LSTM', 'MAE': '15.5602', 'MSE': '268.9962', 'MASE': '61.8665', 'Forecast Bias': '15.56'}


In [21]:
param_bounds = {
    "encoder_type": ["RNN", "GRU", "LSTM"],
    "decoder_type": ["FC"],
    "hidden_size": (32, 128),
    "num_layers": (5, 30),
    "bidirectional": [True, False],
    "decoder_use_all_hidden": [True, False],
}

initial_params = ["RNN", "FC", 32, 5, True, True]
initial_temp = 10

def objective(params):
    encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden = params

    encoder_config = RNNConfig(
        input_size=len(columns_to_select),
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional
    )

    decoder_config = RNNConfig(
        input_size=len(columns_to_select),
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional
    )

    rnn2fc_config = Seq2SeqConfig(
        encoder_type=encoder_type,
        decoder_type=decoder_type,
        encoder_params=encoder_config,
        decoder_params={"window_size": WINDOW, "horizon": HORIZON},
        decoder_use_all_hidden=decoder_use_all_hidden,
        learning_rate=1e-3,
    )

    model = Seq2SeqModel(rnn2fc_config)

    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )

    trainer.fit(model, datamodule)

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean

    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    return np.mean(np.abs(actuals - predictions))

best_cost, best_params, cost_history = simulated_annealing(
    objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate
)

logging.info(f"Best Parameters: {best_params}, Best Cost: {best_cost}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(cost_history) + 1), y=cost_history, mode="lines"))
fig.update_layout(
    title="Simulated Annealing Optimization Seq2Seq for Hyperparameter Tuning",
    xaxis_title="Iteration",
    yaxis_title="Best Cost",
    template="plotly_white",
)

fig.show()

encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden = best_params

encoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional
)

decoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional
)

rnn2fc_config = Seq2SeqConfig(
    encoder_type=encoder_type,
    decoder_type=decoder_type,
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=decoder_use_all_hidden,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)
algorithm_name = f"Optimized Seq2Seq {encoder_type} (SA)"
metric_record, pred_df = train_and_evaluate(model, rnn2fc_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

shutil.rmtree("lightning_logs")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type    | Params
------------------------------------
0 | encoder | RNN     | 28.5 K
1 | decoder | Linear  | 449   
2 | loss    | MSELoss | 0     
------------------------------------
28.9 K    Trainable params
0         Non-trainable params
28.9 K    Total params
0.116     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-22 21:55:42,223 - INFO - {'Algorithm': 'Optimized Seq2Seq RNN (SA)', 'MAE': '10.6560', 'MSE': '129.5150', 'MASE': '42.3674', 'Forecast Bias': '10.66'}


In [22]:
# Now for the new set of experiments with PINNs data
pinn_features = [
    "covidOccupiedMVBeds",
    "hospitalCases_rolling_7_mean",
    "hospitalCases_rolling_7_std",
    "newAdmissions_rolling_7_mean",
    "newAdmissions_rolling_7_std",
    "new_confirmed_rolling_7_mean",
    "new_confirmed_rolling_7_std",
    "new_deceased_rolling_7_mean",
    "new_deceased_rolling_7_std",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
    "susceptible",
    "active_cases",
    "recovered",
    "cumulative_deceased",
]

pinn_sample_df = merged_data[pinn_features]
pinn_sample_df = to_float32(pinn_sample_df)
pinn_cols = list(pinn_sample_df.columns)
pinn_cols.remove("covidOccupiedMVBeds")
pinn_sample_df = pinn_sample_df[pinn_cols + ["covidOccupiedMVBeds"]]

pinn_datamodule = TimeSeriesDataModule(
    data=pinn_sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7,  # 7 days window
    horizon=1,  # single step
    normalize="global",  # normalizing the data
    batch_size=32,
    num_workers=0,
)
pinn_datamodule.setup()

# Initialize pred_df for PINNs experiments
pinn_pred_df = test.copy()

In [23]:
# Function to train and evaluate models with PINNs data
def train_and_evaluate_pinns(model, rnn_config, pinn_datamodule, test, train, algorithm_name, metric_record, pinn_pred_df):
    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )
    trainer.fit(model, pinn_datamodule)

    predictions = trainer.predict(model, pinn_datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * pinn_datamodule.train.std + pinn_datamodule.train.mean
    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    metrics = {
        "Algorithm": algorithm_name,
        "MAE": mae(actuals, predictions),
        "MSE": mse(actuals, predictions),
        "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
        "Forecast Bias": forecast_bias(actuals, predictions),
    }

    value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
    metrics = {
        key: format_.format(value)
        for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
    }

    pred_df_ = pd.DataFrame({algorithm_name: predictions}, index=test.index)
    pinn_pred_df = pinn_pred_df.join(pred_df_)
    metric_record.append(metrics)
    logging.info(metrics)
    return metric_record, pinn_pred_df


In [24]:
# Train Vanilla RNN with PINNs data
rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=len(pinn_features),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "PINNs Vanilla RNN"
metric_record, pinn_pred_df = train_and_evaluate_pinns(
    model,
    rnn_config,
    pinn_datamodule,
    test,
    train,
    algorithm_name,
    metric_record,
    pinn_pred_df,
)

# Train Vanilla LSTM with PINNs data
rnn_config = SingleStepRNNConfig(
    rnn_type="LSTM",
    input_size=len(pinn_features),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "PINNs Vanilla LSTM"
metric_record, pinn_pred_df = train_and_evaluate_pinns(
    model,
    rnn_config,
    pinn_datamodule,
    test,
    train,
    algorithm_name,
    metric_record,
    pinn_pred_df,
)

# Train Vanilla GRU with PINNs data
rnn_config = SingleStepRNNConfig(
    rnn_type="GRU",
    input_size=len(pinn_features),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "PINNs Vanilla GRU"
metric_record, pinn_pred_df = train_and_evaluate_pinns(
    model,
    rnn_config,
    pinn_datamodule,
    test,
    train,
    algorithm_name,
    metric_record,
    pinn_pred_df,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/olarinoyem/Research/Multivate-forecasting/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 10.3 K
1 | fc   | Linear  | 33    
2 | loss | MSELoss | 0     
---------------------------------
10.3 K    Trainable params
0         Non-trainable params
10.3 K    Total params
0.041     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-22 21:55:48,961 - INFO - {'Algorithm': 'PINNs Vanilla GRU', 'MAE': '4494.1494', 'MSE': '20202192.7338', 'MASE': '17868.4307', 'Forecast Bias': '4494.15'}


In [25]:
# Train Optimized GRU (SA) with PINNs data
rnn_config = SingleStepRNNConfig(
    rnn_type=best_params[0],
    input_size=len(pinn_features),
    hidden_size=best_params[1],
    num_layers=best_params[2],
    bidirectional=best_params[3],
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = f"PINNs Optimized {rnn_config.rnn_type} (SA)"
metric_record, pinn_pred_df = train_and_evaluate_pinns(
    model,
    rnn_config,
    pinn_datamodule,
    test,
    train,
    algorithm_name,
    metric_record,
    pinn_pred_df,
)


ValidationError: Value 'FC' of type 'numpy.str_' could not be converted to Integer
    full_key: hidden_size
    object_type=SingleStepRNNConfig

In [ ]:
# Train Seq2Seq with PINNs data
rnn2fc_config = Seq2SeqConfig(
    encoder_type="LSTM",
    decoder_type="FC",
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=True,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)
algorithm_name = "PINNs Seq2Seq LSTM"
metric_record, pinn_pred_df = train_and_evaluate_pinns(
    model,
    rnn2fc_config,
    pinn_datamodule,
    test,
    train,
    algorithm_name,
    metric_record,
    pinn_pred_df,
)


In [ ]:
# Train Optimized Seq2Seq (SA) with PINNs data
encoder_config = RNNConfig(
    input_size=len(pinn_features),
    hidden_size=best_params[1],
    num_layers=best_params[2],
    bidirectional=best_params[3]
)

decoder_config = RNNConfig(
    input_size=len(pinn_features),
    hidden_size=best_params[1],
    num_layers=best_params[2],
    bidirectional=best_params[3]
)

rnn2fc_config = Seq2SeqConfig(
    encoder_type=encoder_type,
    decoder_type=decoder_type,
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=decoder_use_all_hidden,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)
algorithm_name = f"PINNs Optimized Seq2Seq {encoder_type} (SA)"
metric_record, pinn_pred_df = train_and_evaluate_pinns(
    model,
    rnn2fc_config,
    pinn_datamodule,
    test,
    train,
    algorithm_name,
    metric_record,
    pinn_pred_df,
)


In [ ]:
# Save metrics
metric_df = pd.DataFrame(metric_record)
metric_df[["MAE", "MSE", "MASE", "Forecast Bias"]] = metric_df[
    ["MAE", "MSE", "MASE", "Forecast Bias"]
].astype("float32")

final_metric_file = f"reports/metrics/{selected_area}_final_metrics.csv"
metric_df.to_csv(final_metric_file, index=False)

# Save predictions
forecast_columns = [
    "Vanilla RNN", 
    "Vanilla LSTM", 
    "Vanilla GRU", 
    "Optimized GRU (SA)", 
    "Seq2Seq LSTM", 
    "Optimized Seq2Seq RNN (SA)",
    "PINNs Vanilla RNN",
    "PINNs Vanilla LSTM",
    "PINNs Vanilla GRU",
    f"PINNs Optimized {best_params[0]} (SA)",
    "PINNs Seq2Seq LSTM",
    f"PINNs Optimized Seq2Seq {encoder_type} (SA)"
]

forecast_display_names = [
    "Vanilla RNN", 
    "Vanilla LSTM", 
    "Vanilla GRU", 
    "Optimized GRU (SA)", 
    "Seq2Seq LSTM", 
    "Optimized Seq2Seq RNN (SA)",
    "PINNs Vanilla RNN",
    "PINNs Vanilla LSTM",
    "PINNs Vanilla GRU",
    f"PINNs Optimized {best_params[0]} (SA)",
    "PINNs Seq2Seq LSTM",
    f"PINNs Optimized Seq2Seq {encoder_type} (SA)"
]

comparison_fig = plot_forecast(
    pred_df,
    forecast_columns,
    selected_area,
    forecast_display_names,
    save_path=f"reports/figures/{selected_area}_forecast_comparison.pdf",
)
comparison_fig.show()

In [ ]:
pred_df.head(2)

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,hospitalCases_rolling_7_mean,hospitalCases_rolling_7_std,newAdmissions_rolling_7_mean,newAdmissions_rolling_7_std,new_confirmed_rolling_7_mean,...,exposed,active_cases,recovered,cumulative_deceased,Vanilla RNN,Vanilla LSTM,Vanilla GRU,Optimized RNN (SA),Seq2Seq LSTM,Optimized Seq2Seq RNN (SA)
date,,,,,,,,,,,,,,,,,,,,,
2021-04-06,22.323374,2588.0,183,2709.0,22.0,2773.285714,174.717595,184.142857,20.651300,2523.000000,...,79550.64,74167.45,3676206.0,112096.97,24.437866,29.06958,26.936768,7.053711,29.291626,34.949585
2021-04-07,21.984093,2486.0,189,2715.0,24.0,2687.857143,140.274051,179.000000,11.015141,2438.714286,...,77862.80,72701.23,3680953.5,112115.51,25.661865,28.96460,26.852417,7.052490,29.174194,33.756104


In [ ]:
pred_df.to_csv(f"reports/results/{selected_area}_predictions.csv")